<a href="https://colab.research.google.com/github/agi2019/ppi-gci/blob/main/tutorials/02%20-%20model%20calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Model calibration</center>

Prepared by Omar A. Guerrero (oguerrero@turing.ac.uk, <a href="https://twitter.com/guerrero_oa">@guerrero_oa</a>)

In this tutorial I will calibrate the free parameters of PPI's model. First, I will load all the data that I have prepared in the previous tutorials. Then, I extract the relevant information and put it in adequate data structures. Finally, I run the calibration function and save the results with the parameter values.

## Importing Python libraries to manipulate data

In [1]:
import pandas as pd
import numpy as np

Select Scenario

In [2]:
#scenario = '_scenario1'
#scenario = '_scenario2'
scenario = '_scenario3'

## Importing PPI functions

In this tutorial, I will import the PPI source code directly from its repository. This means that I will place a request to GitHub, download the `policy_priority_inference.py` file, and copy it locally into the folder where these tutorials are saved. Then, I will import PPI. This approach is useful if you want to run this tutorial in a cloud computing service.

An alternative would be to manually copy the `policy_priority_inference.py` file into the folder where this tutorial is located.

In [3]:
import policy_priority_inference as ppi

## Load data

In [4]:
df_exp = pd.read_csv('C:/Users/rha005/OneDrive - CSIRO/Documents/7. CSIRO/00. Work in Progress - ON Office/Week 9-10/ppi-gci-main-indicators/tutorials/clean_data'+scenario+'/data_expenditure.csv')
len(df_exp)

63

### Indicators

In [5]:
df_indis = pd.read_csv('C:/Users/rha005/OneDrive - CSIRO/Documents/7. CSIRO/00. Work in Progress - ON Office/Week 9-10/ppi-gci-main-indicators/tutorials/clean_data'+scenario+'/data_indicators.csv').set_index("seriesCode")
print(len(df_indis))
seriesCodes = df_exp['seriesCode'].values
df_indis = df_indis.loc[seriesCodes]  # Reorder sesuai Bs

Bs = df_exp.drop(columns='seriesCode').values
N = len(df_indis) # number of indicators
I0 = df_indis.I0.values # initial values
IF = df_indis.IF.values # final values
success_rates = df_indis.successRates.values # success rates
R = df_indis.instrumental.astype(bool).values # instrumental indicators
qm = df_indis.qm.values # quality of monitoring
rl = df_indis.rl.values # quality of the rule of law
indis_index = {code: i for i, code in enumerate(df_indis.index)}
seriesCodes = df_exp['seriesCode'].values


63


### Interdependency network

In [6]:
df_net = pd.read_csv('C:/Users/rha005/OneDrive - CSIRO/Documents/7. CSIRO/00. Work in Progress - ON Office/Week 9-10/ppi-gci-main-indicators/tutorials/clean_data'+scenario+'/data_network.csv')

valid_codes = set(df_indis.index)
df_net = df_net[df_net['origin'].isin(valid_codes)]
df_net = df_net[df_net['destination'].isin(valid_codes)]

A = np.zeros((len(indis_index), len(indis_index)))

for _, row in df_net.iterrows():
    i = indis_index[row['origin']]
    j = indis_index[row['destination']]
    A[i, j] = row['weight']


### Budget

In [7]:
df_exp

,seriesCode,0,1,2,3,4,5,6,7,8,...,50,51,52,53,54,55,56,57,58,59
0,gci11_Lolsafety,1.178240e+08,1.178240e+08,1.178240e+08,1.178240e+08,1.178240e+08,1.178240e+08,1.178240e+08,1.178240e+08,1.178240e+08,...,1.479349e+09,1.479349e+09,1.479349e+09,1.479349e+09,1.479349e+09,1.479349e+09,1.479349e+09,1.479349e+09,1.479349e+09,1.479349e+09
1,gci11_Lforgery,8.751650e+08,8.751650e+08,8.751650e+08,8.751650e+08,8.751650e+08,8.751650e+08,8.751650e+08,8.751650e+08,8.751650e+08,...,4.982700e+07,4.982700e+07,4.982700e+07,4.982700e+07,4.982700e+07,4.982700e+07,4.982700e+07,4.982700e+07,4.982700e+07,4.982700e+07
2,gci11_Lonline,2.373030e+08,2.373030e+08,2.373030e+08,2.373030e+08,2.373030e+08,2.373030e+08,2.373030e+08,2.373030e+08,2.373030e+08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,gci12_Rpdp,1.683348e+09,1.683348e+09,1.683348e+09,1.683348e+09,1.683348e+09,1.683348e+09,1.683348e+09,1.683348e+09,1.683348e+09,...,2.884810e+08,2.884810e+08,2.884810e+08,2.884810e+08,2.884810e+08,2.884810e+08,2.884810e+08,2.884810e+08,2.884810e+08,2.884810e+08
4,gci12_Rprivacy,1.324220e+08,1.324220e+08,1.324220e+08,1.324220e+08,1.324220e+08,1.324220e+08,1.324220e+08,1.324220e+08,1.324220e+08,...,6.736580e+08,6.736580e+08,6.736580e+08,6.736580e+08,6.736580e+08,6.736580e+08,6.736580e+08,6.736580e+08,6.736580e+08,6.736580e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,gci52_Mcapacitydev,9.716000e+06,9.716000e+06,9.716000e+06,9.716000e+06,9.716000e+06,9.716000e+06,9.716000e+06,9.716000e+06,9.716000e+06,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
59,gci53_Mlat,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
60,gci54_PPPdomestic,1.577239e+09,1.577239e+09,1.577239e+09,1.577239e+09,1.577239e+09,1.577239e+09,1.577239e+09,1.577239e+09,1.577239e+09,...,4.608430e+08,4.608430e+08,4.608430e+08,4.608430e+08,4.608430e+08,4.608430e+08,4.608430e+08,4.608430e+08,4.608430e+08,4.608430e+08
61,gci54_PPPforeign,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,...,1.170940e+08,1.170940e+08,1.170940e+08,1.170940e+08,1.170940e+08,1.170940e+08,1.170940e+08,1.170940e+08,1.170940e+08,1.170940e+08


In [8]:
instrumental_indices = np.where(R)[0]

# Buat peta program yang digunakan
used_programs = sorted(set(instrumental_indices))
old2new = {old: new for new, old in enumerate(used_programs)}

# Bangun ulang B_dict dan Bs_filtered
B_dict = {i: [old2new[i]] for i in instrumental_indices}
Bs_filtered = Bs[used_programs, :]

### Budget-indicator mapping

In [9]:
#df_rela = pd.read_csv('C:/Users/rha005/OneDrive - CSIRO/Documents/7. CSIRO/00. Work in Progress - ON Office/Week 9-10/ppi-gci-main-indicators/tutorials/clean_data'+scenario+'/data_relational_table.csv')

#B_dict = {} # PPI needs the relational table in the form of a Python dictionary
#for index, row in df_rela.iterrows():
#    B_dict[indis_index[row.seriesCode]] = [programme for programme in row.values[1::][row.values[1::].astype(str)!='nan']]

In [10]:
print("Jumlah baris I0:", len(I0))
print("Jumlah baris IF:", len(IF))
print("Jumlah indikator instrumental:", R.sum())
print("Jumlah baris Bs_filtered:", len(Bs_filtered))

Jumlah baris I0: 63
Jumlah baris IF: 63
Jumlah indikator instrumental: 57
Jumlah baris Bs_filtered: 57


## Calibrate

Now I run the calibration function to show that it works. Before that, let me explain a couple of new inputs that the user needs to provide:

* <strong>threshold</strong>: How well should the model be fit.
* <strong>parallel_processes</strong>: The number of processes (workers) to be ran in parallel.
* <strong>verbose</strong>: Whether to print or not the outputs as the calibration progresses.
* <strong>low_precision_counts</strong>: The number of iterations that use few Monte Carlo simulations.

The <strong>threshold</strong> parameter indicates the quality of the goodness of fit. More specifically, how good should the worst-fitted indicator be. The best possible fit is close to 1, but cannot be exactly 1 due to the stochasticity of the model. The higher the threshold, the mode Monte Carlo simulations are needed and, thus, the more time and computational resources are needed to complete the calibration.

Parameter <strong>parallel_processes</strong> is used to enhance efficiency. Since each Monte Carlo simulation is independent of each other, this workload can be distributed across multiple cores or processors. Today, most personal devices have the capability of handling this distributed load, so here I show how to calibrate the model using 4 parallel processes. It is recommended that you know how many cores or processors your equipment has, and that <strong>parallel_processes</strong> does not exceed that number. Otherwise, the performance of the calibration may be sub-optimal.

Finally, the <strong>low_precision_counts</strong> parameter helps accelerating the calibration. At the beginning of the calibration, the algorithm proposes a random set of parameters for the model. Because this proposal is unrelated to the true parameters, the errors tend to be large. In the presence of large errors, one can improve the goodness of fit without needing too much precision in each evaluation (i.e., without running too many Monte Carlo simulations). Hence, this parameter determines how many low-precision iterations of the algorithm should be run before proceeding to the high-precision ones. This accelerates the calibration procedure substantially.

In [11]:
T = Bs.shape[1]
parallel_processes = 4 # number of cores to use
threshold = 0.95 # the quality of the calibration (I choose a medium quality for illustration purposes)
low_precision_counts = 100 # number of low-quality iterations to accelerate the calibration

parameters = ppi.calibrate(I0, IF, success_rates, A=A, R=R, qm=qm, rl=rl, Bs=Bs_filtered, B_dict=B_dict,
              T=T, threshold=threshold, parallel_processes=parallel_processes, verbose=True,
             low_precision_counts=low_precision_counts)

Iteration: 1 .    Worst goodness of fit: -573814003519.8511
Iteration: 2 .    Worst goodness of fit: -140854035626.10886
Iteration: 3 .    Worst goodness of fit: -35709199365.31692
Iteration: 4 .    Worst goodness of fit: -10154166001.510971
Iteration: 5 .    Worst goodness of fit: -2064997729.0134273
Iteration: 6 .    Worst goodness of fit: -575863299.4180259
Iteration: 7 .    Worst goodness of fit: -133435986.29984
Iteration: 8 .    Worst goodness of fit: -35370795.63406516
Iteration: 9 .    Worst goodness of fit: -10084595.42881618
Iteration: 10 .    Worst goodness of fit: -2208397.6450256007
Iteration: 11 .    Worst goodness of fit: -528964.2327730223
Iteration: 12 .    Worst goodness of fit: -142537.5892837949
Iteration: 13 .    Worst goodness of fit: -33482.90442882983
Iteration: 14 .    Worst goodness of fit: -8603.19360728441
Iteration: 15 .    Worst goodness of fit: -1956.745728417224
Iteration: 16 .    Worst goodness of fit: -486.6527047837242
Iteration: 17 .    Worst goodnes

## Calibration outputs

The output of the calibration function is a matrix with the following columns:

* <strong>alpha</strong>: the parameters related to structural constraints
* <strong>alpha_prime</strong>: the parameters related to structural costs
* <strong>beta</strong>: the parameters related to the probability of success
* <strong>T</strong>: the number of simulation periods
* <strong>error_alpha</strong>: the errors associated to the parameters $\alpha$ and $\alpha'$
* <strong>error_beta</strong>: the errors associated to the parameters $\beta$
* <strong>GoF_alpha</strong>: the goodness-of-fit associated to the parameters $\alpha$ and $\alpha'$
* <strong>GoF_beta</strong>: the goodness-of-fit associated to the parameters $\beta$

The top row of this matrix contains the column names, so I just need to transform these data into a DataFrame to export it.

In [12]:
df_params = pd.DataFrame(parameters[1::], columns=parameters[0])

In [13]:
df_params

,alpha,alpha_prime,beta,T,error_alpha,error_beta,GoF_alpha,GoF_beta
0,0.01719371058796743,5.10487377673869e-13,4.638724849902477e-10,60,-0.0011177507852457857,-0.0005551030348941582,0.9776449842950843,0.9888979393021169
1,0.0036203251516096465,6.081796776681211e-05,3.8761149691731885e-09,nan,0.000770063308991209,0.0034681846615375878,0.9900089691828319,0.9930636306769248
2,0.0023326595035337338,5.053880569260366e-05,1.2330403295832136e-09,nan,-0.0004750985928640983,-0.0013988529240483993,0.9899042165006207,0.9972022941519032
3,0.010919860898180193,0.0001700792078119376,3.515555997325643e-09,nan,0.0017911290600888252,0.0016047156111795302,0.993059362426415,0.9967905687776409
4,0.010727493166994817,0.00016368887577899755,5.380339317745586e-09,nan,-0.003056878309244393,-0.00084630981593381,0.9881545752767397,0.9983073803681324
...,...,...,...,...,...,...,...,...
58,0.016904992479061385,2.364531346587313e-09,3.39717989423713e-09,nan,0.0013085431413228932,0.0003706067738961927,0.9738291371735421,0.9925878645220761
59,0.016244199954605687,0.0008977794735395071,1.2179233462075454,nan,-0.00809033510624202,-0.0010601435877858334,0.985408304488289,0.998233094020357
60,0.016759305738557218,7.998618960462577e-05,1.4825949321069911e-09,nan,0.009308251441213167,0.005647190882863695,0.9832116755371785,0.9905880151952272
61,0.021115077205759773,0.0006457865145576095,1.366699746991253e-08,nan,0.0024985153492251833,-0.0022021623442890492,0.9954936878721996,0.9963297294261849


In [14]:
print("Jumlah baris parameter:", len(df_params))
print("Jumlah indikator instrumental:", R.sum())

Jumlah baris parameter: 63
Jumlah indikator instrumental: 57


## Save parameters data

In [15]:
df_params.to_csv('C:/Users/rha005/OneDrive - CSIRO/Documents/7. CSIRO/00. Work in Progress - ON Office/Week 9-10/ppi-gci-main-indicators/tutorials/clean_data'+scenario+'/parameters.csv', index=False)